In [3]:
# Edge AI Image Classifier for Recyclables
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
#from tensorflow.lite import TFLiteConverter
import os

# Load base model
base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
base_model.trainable = False

# 1. Create a lightweight CNN model
model = models.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(4, activation='softmax')  # 4 recyclable categories
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Dummy data generator for simulation
datagen = ImageDataGenerator(rescale=1./255)
train_data = datagen.flow_from_directory(
    'recyclable_data/',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

# Train
model.fit(train_data, epochs=5)

# 2. Convert to TensorFlow Lite
converter = TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 3. Save the model
with open('recyclable_classifier.tflite', 'wb') as f:
    f.write(tflite_model)

print("Edge AI model successfully converted to TFLite format!")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: 'recyclable_data/'